In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib, json
from sentence_transformers import SentenceTransformer
import faiss

app = FastAPI()

# load models / index
intent_pipe = joblib.load("models/intent_baseline.joblib")  # or TF model
sbert = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
index = faiss.read_index("data/faiss_index.idx")
with open("data/faiss_meta.json",'r',encoding='utf8') as f:
    meta = json.load(f)
# docs list must be loaded similarly

class Query(BaseModel):
    text: str

@app.post("/query")
def query(q: Query):
    text = q.text
    # 1) intent
    intent = intent_pipe.predict([text])[0]
    # 2) if high-confidence FAQ -> return template
    # otherwise retrieval
    q_emb = sbert.encode([text], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    D,I = index.search(q_emb, 5)
    top_passages = [docs[i] for i in I[0]]
    # 3) generate answer (use local generator or API)
    answer = generate_answer(text, top_passages)  # from rag_generate.py
    return {"intent": intent, "answer": answer, "sources": [meta[i]['source'] for i in I[0]]}
